### Building the Image Dataset

In [3]:
import os


Now, display a batch of 8 negative and 8 positive pictures. You can rerun the cell to see a fresh batch each time:

#### Data Preprocessing

Now, let's use `keras.preprocessing.image.ImageDataGenerator` class to create our train and validation dataset and normalize our data. 

It's important to normalize our data because data going into our CNN to improve its overall performance. We will use the `rescale` parameter to scale our image pixel values from [0, 255] to [0,1].

In each generator, we specify the source directory of our images, the classes, the input image size, the batch size (how many images to process at once), and class mode.

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'train',  # This is the source directory for training images
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=120)

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'test',  # This is the source directory for training images
        target_size = (200, 200),  # All images will be resized to 200x200
        batch_size = 19,
        shuffle=False)

Found 5232 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


## Building the Model from Scratch

But before we continue, let's start defining the model:

Step 1 will be to import tensorflow.

In [5]:
import tensorflow as tf

Let's then add a Flatten layer that flattens the input image, which then feeds into the next layer, a Dense layer, or fully-connected layer, with 128 hidden units. Finally, because our goal is to perform binary classification, our final layer will be a sigmoid, so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is of class 1

In [6]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (200,200,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(3, activation=tf.nn.softmax)])

Metal device set to: Apple M2 Max


2023-03-07 17:22:59.599907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-07 17:22:59.599929: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


The model.summary() method call prints a summary of the NN 

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 120000)            0         
                                                                 
 dense (Dense)               (None, 128)               15360128  
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 15,360,515
Trainable params: 15,360,515
Non-trainable params: 0
_________________________________________________________________


The "output shape" column shows the transformation of the dimensions of each layer as a result of the convolution and max pooling - convolution will reduce the layer size by a bit due to padding, and max pooling will halve the output size.

Next, we'll configure the specifications for model training. We will train our model with the `binary_crossentropy` loss. We will use the `Adam` optimizer. [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) is a sensible optimization algorithm because it automates learning-rate tuning for us (alternatively, we could also use [RMSProp](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) or [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad) for similar results). We will add accuracy to `metrics` so that the model will monitor accuracy during training

In [8]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

### Training

In [9]:
History = model.fit(train_generator,
      steps_per_epoch=2,
      epochs=45,
      validation_data = validation_generator,
      validation_steps= 10)

Epoch 1/45


2023-03-07 17:23:00.501275: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-07 17:23:00.657413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 47.4388 - accuracy: 0.4375

2023-03-07 17:23:02.204014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 970ms/step - loss: 47.4388 - accuracy: 0.4375 - val_loss: 132.3265 - val_accuracy: 0.0000e+00
Epoch 2/45
2/2 [==============================] - 2s 1s/step - loss: 88.0705 - accuracy: 0.2333 - val_loss: 32.0712 - val_accuracy: 0.0000e+00
Epoch 3/45
2/2 [==============================] - 2s 1s/step - loss: 51.5722 - accuracy: 0.3417 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/45
2/2 [==============================] - 2s 1s/step - loss: 68.4194 - accuracy: 0.4292 - val_loss: 7.3794 - val_accuracy: 0.1421
Epoch 5/45
2/2 [==============================] - 2s 1s/step - loss: 52.6225 - accuracy: 0.2958 - val_loss: 0.3966 - val_accuracy: 0.9000
Epoch 6/45
2/2 [==============================] - 2s 1s/step - loss: 29.6587 - accuracy: 0.6000 - val_loss: 5.8963e-05 - val_accuracy: 1.0000
Epoch 7/45
2/2 [==============================] - 2s 1s/step - loss: 15.1045 - accuracy: 0.5667 - val_loss: 24.6099 - val_accuracy: 0.0579
Epoch 8/45
2/2 [======

Save the model

Note: to load model on raspberry pi see: https://www.tensorflow.org/tutorials/keras/save_and_load

"Reload a fresh Keras model from the saved model:

new_model = tf.keras.models.load_model('saved_model/my_model') 

..."

In [10]:
#!mkdir -p saved_model
#model.save('saved_model/my_model')


## Accuracy



Let's evaluate the accuracy of our model:

In [11]:
acc = History.history['accuracy'][-1]
val_acc = History.history['val_accuracy'][-1]
loss = History.history['loss'][-1]
val_loss = History.history['val_loss'][-1]

print("Training accuracy: ", acc)
print("Training loss: ", loss)

print("Validation accuracy: ", val_acc)
print("Validation loss: ", val_loss)

Training accuracy:  0.5916666984558105
Training loss:  2.1980926990509033
Validation accuracy:  0.9789474010467529
Validation loss:  0.059243205934762955
